# Reservoir Management

## Problem Description

From [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470)

> The water supply system under consideration is that of a water supply operator based
in a town with a population of about 25,000 inhabitants, located in Eastern Poland. **The
main parts of the system are wells, pumps, a reservoir tank, and the distribution pipeline
network.**

> Supplied water is groundwater pumped from 7 wells. The capacities and values of
the electric power of the pumps are presented in Table 1. **Water is pumped from the wells
to a single reservoir tank with the capacity of Vmax = 1500 m3
(the maximal volume of
stored water)**.

> **The demand for water varies over time**. The outflow of water from the reservoir
tank via the distribution network to customers is a continuous process. For practical
reasons, **predictions of demand are made for 24 one-hour timeslots**.

> Controlling the pumps must obey the following requirements. **The pumps can operate
with their nominal capacities only**, and the amount of water pumped by any pump depends
on the time of operation only. **Each pump must operate for at least one hour per day.**
Additionally, **at least one well and the pump integrated with it must be kept as a reserve at
any moment of the day**. The water inside the tank should be replaced at least once per day.
During standard operational conditions, **the volume of water in the reservoir tank cannot
be less than Vmin = 523.5 m3**.
It is the firefighting reserve, which is kept in order to satisfy
an extra demand when a fire is extinguished by using water supplied from hydrants.

> ... the supplier of electric
power does not use the same rate per MWh in its pricing policy all day long. Instead, it
uses three tariff levels ...

## Problem instance data

From [Kowalik P, Rzemieniak M. Binary Linear Programming as a Tool of Cost Optimization for a Water Supply Operator. Sustainability. 2021 13(6):3470.](https://doi.org/10.3390/su13063470)

In [1]:
%%file pumps.csv
id capacity power_consumption
1 75 15
2 133 37
3 157 33
4 176 33
5 59 22
6 69 33
7 120 22

Overwriting pumps.csv


In [2]:
%%file tariff.csv
start end price
16 21 336.00
7 13 283.00
0 7 169.00
13 16 169.00
21 24 169.00

Overwriting tariff.csv


In [3]:
%%file schedule.csv
time water_demand power_price
1 44.62 169
2 31.27 169
3 26.22 169
4 27.51 169
5 31.50 169
6 46.18 169
7 69.47 169
8 100.36 283
9 131.85 283
10 148.51 283
11 149.89 283
12 142.21 283
13 132.09 283
14 129.29 169
15 124.06 169
16 114.68 169
17 109.33 336
18 115.76 336
19 126.95 336
20 131.48 336
21 138.86 336
22 131.91 169
23 111.53 169
24 70.43 169

Overwriting schedule.csv


In [4]:
%%file reservoir.csv
Vmin Vmax Vinit
523.5 1500 550

Overwriting reservoir.csv


## Load data

In [5]:
import itertools as it
import functools as ft
import collections as coll
import random, json, string, sys, pickle, shutil, string

import numpy as np
import pandas as pd
import networkx as nx
from cytoolz import curried as tl
import janitor

from icecream import ic
from prettyprinter import pprint as pp, install_extras

import dimod

In [6]:
pumps = pd.read_csv("pumps.csv", sep=' ').set_index('id', drop=False)
schedule = pd.read_csv("schedule.csv", sep=' ').set_index('time', drop=False)
tariff = pd.read_csv("tariff.csv", sep=' ')
reservoir = pd.read_csv("reservoir.csv", sep=' ').loc[0]

## Define domain language

In [7]:
Sum = dimod.quicksum

def capacity(pump):
    return pumps.capacity[pump]

def power_consumption(pump):
    return pumps.power_consumption[pump]

def power_price(time):
    return schedule.power_price[time]/1000

def water_demand(time):
    return schedule.water_demand[time]

def is_running(pump, time):
    return dimod.Binary(f"pump{pump}_time{time}")

def volume(time): 
    return (
        dimod.Binary(f"volume_time{time}")
        if time >= schedule.time.min()
        else reservoir.Vinit
    )

def inflow(time):
    return Sum(
            pumps.capacity[pump] * is_running(pump, time)
            for pump in pumps.id
        )

## Construct model

In [8]:
model = dimod.CQM()

### Define objective

#### *Minimize power costs*

In [9]:
model.set_objective(
    Sum(
        power_price(time)
        * Sum(
            power_consumption(pump) * is_running(pump, time)
            for pump in pumps.id
        )
        for time in schedule.time
    )
)

### Add constraints

#### *Each pump must operate for at least one hour per day*

In [10]:
for pump in pumps.id:
    model.add_constraint(
        Sum(
            is_running(pump, time) for time in schedule.time
        ) >= 1,
        f"pump_{pump} must operate for at least one hour per day")

#### *At least one well and the pump integrated with it must be kept as a reserve at any moment of the day*

In [11]:
for time in schedule.time:
    model.add_constraint(
        Sum(
            is_running(pump, time) for pump in pumps.id
        ) <= pumps.shape[0] - 1,
        f"at least one pump is in reserve at time {time}" )

#### *The water inside the tank should be replaced at least once per day* ??? (Not addressed in paper. Meaning?)

In [12]:
model.add_constraint(volume(schedule.time.max()) >= reservoir.Vinit);

#### *The volume of water in the reservoir tank must always be between Vmin and Vmax*

In [13]:
for time in schedule.time:
    model.add_constraint(
        volume(time-1) + inflow(time) - volume(time) == water_demand(time),
        f"reservoir volume at time_{time} = volume at time_{time-1} + inflow at time_{time} - water demand at time_{time}"
    )
    model.add_constraint(
        volume(time) >= reservoir.Vmin,
        f"reservoir volume at time {time} >= Vmin"
    )
    model.add_constraint(
        volume(time) <= reservoir.Vmax,
        f"reservoir volume at time {time} <= Vmax"
    )

## Inspect model

In [14]:
model._STR_MAX_DISPLAY_ITEMS = 1000

In [15]:
print(model)

Constrained quadratic model: 192 variables, 104 constraints, 768 biases

Objective
  2.535*Binary('pump1_time1') + 6.253*Binary('pump2_time1') + 5.577*Binary('pump3_time1') + 5.577*Binary('pump4_time1') + 3.7180000000000004*Binary('pump5_time1') + 5.577*Binary('pump6_time1') + 3.7180000000000004*Binary('pump7_time1') + 2.535*Binary('pump1_time2') + 6.253*Binary('pump2_time2') + 5.577*Binary('pump3_time2') + 5.577*Binary('pump4_time2') + 3.7180000000000004*Binary('pump5_time2') + 5.577*Binary('pump6_time2') + 3.7180000000000004*Binary('pump7_time2') + 2.535*Binary('pump1_time3') + 6.253*Binary('pump2_time3') + 5.577*Binary('pump3_time3') + 5.577*Binary('pump4_time3') + 3.7180000000000004*Binary('pump5_time3') + 5.577*Binary('pump6_time3') + 3.7180000000000004*Binary('pump7_time3') + 2.535*Binary('pump1_time4') + 6.253*Binary('pump2_time4') + 5.577*Binary('pump3_time4') + 5.577*Binary('pump4_time4') + 3.7180000000000004*Binary('pump5_time4') + 5.577*Binary('pump6_time4') + 3.718000000000